In [1]:
SEED = sum(map(ord, "FOURIER_MSRA_2D"))

In [2]:
import sys

sys.path.append('../../..')

In [3]:
from lib.msra_analytical import AnalyticalLossFunctionAbs

In [4]:
import warnings

warnings.filterwarnings('error')

In [5]:
import numpy

import scipy
import scipy.stats
from scipy.integrate import quad

In [6]:
def fourier_integral1d(cplx_integrand, j, x, alpha):
    def real_integrand(u):
        return scipy.real(cplx_integrand(u, j, x, alpha))

    real_quad = quad(real_integrand, -numpy.inf, numpy.inf, epsrel=1e-4)[0]
    
    return real_quad

In [7]:
class FourierGaussLossFunction(AnalyticalLossFunctionAbs):
    def __init__(self, v_mu, v_sigma, c=None):
        self.__v_mu = numpy.array(v_mu)
        self.__v_sigma = numpy.array(v_sigma).reshape((len(v_mu), len(v_mu)))
        
        self.__minus_1_half_over_pi = -0.5 / numpy.pi
        self.__1_over_pi = 1. / numpy.pi
        
        super(FourierGaussLossFunction, self).__init__(len(v_mu), c)
        
    def moment_generating(self, t, i):
        mu = self.__v_mu[i]
        sigma2 = self.__v_sigma[i, i]
        
        log_part = mu * t + 0.5 * sigma2 * t**2
        return numpy.exp(log_part)
        
    def e(self, m):
        return (self.__v_mu - m).sum()
    
    def g_fourier_integrand(self, u, j, m_j, eta):
        i = 1j

        eta_m_iu = eta - i * u
        
        res = numpy.exp(-eta_m_iu * m_j)
        res *= self.moment_generating(eta_m_iu, j)
        res /= i * (u + i * eta)**3
        
        return res
    
    def g(self, m):
        res = 0.
        eta = 1.5 * numpy.random.rand()
        for i, m_i in enumerate(m):
            continue_bool = True
            while continue_bool:
                try:
                    integral = fourier_integral1d(self.g_fourier_integrand, i, m_i, eta)
                    continue_bool = False
                except scipy.integrate.IntegrationWarning, e:
                    #print "g not converging for x = %s, alpha = %s" % (m_i, eta)
                    eta = 1.5 * numpy.random.rand()
            
            res += self.__1_over_pi * integral
            
        return res
    
    def f_fourier_integrand(self, u, j, m_j, eta):
        i = 1j

        eta_m_iu = eta - i * u
        
        res = numpy.exp(-eta_m_iu * m_j)
        res *= self.moment_generating(eta_m_iu, j)
        res /= (u + i * eta)**2
        
        return res
    
    def f(self, i, m_i):
        continue_bool = True
        eta = 1.5 * numpy.random.rand()
        while continue_bool:
            try:
                integral = fourier_integral1d(self.f_fourier_integrand, i, m_i, eta)
                continue_bool = False
                return self.__minus_1_half_over_pi * integral
            except scipy.integrate.IntegrationWarning, e:
                #print "f not converging for x = %s, alpha = %s" % (m_i, eta)
                eta = 1.5 * numpy.random.rand()
        
    def shortfall_risk(self, m=None):
        m = self._check_argument(m)
        
        sum_e = self.e(m)
        sum_g = self.g(m)
        
        return sum_e + 0.5 * sum_g
    
    def shortfall_risk_jac(self, m):
        m = self._check_argument(m)
        
        res = []        
        for i, m_i in enumerate(m):
            partial_der = 1 + self.f(i, m_i)
            res.append(partial_der)
            
        return numpy.array(res)

In [8]:
mu = [0., 0., 0.]
sigma = [[0.5, 0., 0.], [0., 0.5, 0.], [0., 0., 0.6]]

c = 1.

loss = FourierGaussLossFunction(mu, sigma, c)

In [9]:
from scipy.optimize import minimize

maxiter = 3500

In [10]:
guess = 0. * numpy.ones(loss.dim)

In [11]:
cons = ({'type': 'ineq',
         'fun' : lambda x: loss.ineq_constraint(x),
         'jac' : lambda x: loss.ineq_constraint_jac(x)})

In [12]:
numpy.random.seed(SEED)

res = minimize(loss.objective, guess, 
               jac=loss.objective_jac, 
               constraints=cons, 
               method='SLSQP',
               options={'maxiter': maxiter})

In [13]:
%%timeit

numpy.random.seed(SEED)

res = minimize(loss.objective, guess, 
               jac=loss.objective_jac, 
               constraints=cons, 
               method='SLSQP',
               options={'maxiter': maxiter})

10 loops, best of 3: 85.3 ms per loop


In [14]:
print res
print
print loss.ineq_constraint(res.x)

     fun: -0.45118283330834824
     jac: array([ 1.,  1.,  1.,  0.])
 message: 'Optimization terminated successfully.'
    nfev: 5
     nit: 5
    njev: 5
  status: 0
 success: True
       x: array([-0.16566642, -0.16566642, -0.11984999])

-1.18334224641e-07


In [15]:
res.x.sum()

-0.45118283330834824